<span style="color:black"><font size = 5> Task1 : Database setting</span></font>

<span style="color:black"><font size = 5> Task1.(a) : MongoDB & Image Data </span></font>

In [1]:
#Task1: Database setting

#1.(a) For image data, we store them in MongoDB
from pymongo import MongoClient
import urllib

# MongoDB in Docker
mgdb_host = '127.0.0.1'
mgdb_port = '27017'
mgdb_username = ''
mgdb_password = ''
mgdb_database = 'Project'
mgdb_collection = 'blog'

# Establish Connections
client = MongoClient('mongodb://%s/%s' % (mgdb_host, mgdb_database))
mgdb_database = client[mgdb_database]
mgdb_collection = mgdb_database[mgdb_collection]

In [2]:
#Test whether data input is successful
insert_dict = dict()
insert_dict['machine'] = "photo1"
insert_dict['message'] = 'this is just a simple message'
mgdb_collection.insert_one(insert_dict)
print('Insert success')


Insert success


In [3]:
#Delete the test data
mgdb_collection.delete_one({})
print('Delete success')

Delete success


In [4]:
#Access photos from local machine
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(cv2.resize(img,(256,256)))
    return images

In [5]:
photos_PN = load_images_from_folder('/Users/wuguanxun/Desktop/photos/bird_photos/Psilopogon_nuchalis')
photos_PS = load_images_from_folder('/Users/wuguanxun/Desktop/photos/bird_photos/Pycnonotus_sinensis')

In [6]:
#Start insert photo images into MongoDB

def insert_image(image,species,index):
    insert_dict = dict()
    insert_dict['index'] = index
    insert_dict['image'] = (image/255.).tolist()
    insert_dict['species'] = species
    mgdb_collection.insert_one(insert_dict)
    print('Insert success: ',index)

In [7]:
index = 0

for img in photos_PN:
    insert_image(img,'Psilopogon_nuchalis',index)
    index = index+1
for img in photos_PS:
    insert_image(img,'Pycnonotus_sinensis',index)
    index = index+1

Insert success:  0
Insert success:  1
Insert success:  2
Insert success:  3
Insert success:  4
Insert success:  5
Insert success:  6
Insert success:  7
Insert success:  8
Insert success:  9
Insert success:  10
Insert success:  11
Insert success:  12
Insert success:  13
Insert success:  14
Insert success:  15
Insert success:  16
Insert success:  17
Insert success:  18
Insert success:  19
Insert success:  20
Insert success:  21
Insert success:  22
Insert success:  23
Insert success:  24
Insert success:  25
Insert success:  26
Insert success:  27
Insert success:  28
Insert success:  29
Insert success:  30
Insert success:  31
Insert success:  32
Insert success:  33
Insert success:  34
Insert success:  35
Insert success:  36
Insert success:  37
Insert success:  38
Insert success:  39
Insert success:  40
Insert success:  41
Insert success:  42
Insert success:  43
Insert success:  44
Insert success:  45
Insert success:  46
Insert success:  47
Insert success:  48
Insert success:  49
Insert suc


<span style="color:black"><font size = 5> Task1.(b) For metadata derived from data itself, we store them in MySQL(Skipped)</span></font>

<span style="color:black"><font size = 5> Task2: Data Load, Data Preprocessing</span></font>

In [8]:
#Task2: Data Load, Data Preprocessing

#We want:
#(a.) the training data is able to go through sklearn model:Kmeans 
#(we need to flatten each photo image to a 1D-array of size(256*256*3))

#(b.) the training data is able to be selected
#(an index to pick photo we want)

#(c.) the training data and the validation data are capable of passing through CNN from tensorflow

<span style="color:black"><font size = 5> Task2.(a) : Prepare data for KMeans</span></font>

In [12]:
#2.a Prepare data for KMeans

#Fetch data from MongoDB
import pandas as pd
import numpy as np
df = pd.DataFrame(list(mgdb_collection.find()))
df

,_id,index,image,species
0,64511d0cd7d9538f9b463e0a,0,"[[[0.011764705882352941, 0.3254901960784314, 0...",Psilopogon_nuchalis
1,64511d0dd7d9538f9b463e0b,1,"[[[0.09803921568627451, 0.47843137254901963, 0...",Psilopogon_nuchalis
2,64511d0dd7d9538f9b463e0c,2,"[[[0.11764705882352941, 0.49019607843137253, 0...",Psilopogon_nuchalis
3,64511d0dd7d9538f9b463e0d,3,"[[[0.03137254901960784, 0.13333333333333333, 0...",Psilopogon_nuchalis
4,64511d0dd7d9538f9b463e0e,4,"[[[0.00784313725490196, 0.611764705882353, 0.3...",Psilopogon_nuchalis
...,...,...,...,...
306,64511d43d7d9538f9b463f3c,306,"[[[0.35294117647058826, 0.403921568627451, 0.3...",Pycnonotus_sinensis
307,64511d43d7d9538f9b463f3d,307,"[[[0.6627450980392157, 0.6823529411764706, 0.6...",Pycnonotus_sinensis
308,64511d44d7d9538f9b463f3e,308,"[[[0.12156862745098039, 0.3568627450980392, 0....",Pycnonotus_sinensis
309,64511d44d7d9538f9b463f3f,309,"[[[0.1411764705882353, 0.3764705882352941, 0.3...",Pycnonotus_sinensis


In [38]:
#We flatten a (256,256,3) array into a 1D-array with size (256*256*3)
#and combine these arrays into a array
from sklearn.model_selection import train_test_split
X = np.array([np.array(img).flatten() for img in df['image']])
train_X, test_X, train_y, test_y = train_test_split(X,df.species,random_state = 100,test_size = 0.3)

In [43]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters =  k)
kmeans.fit(train_X)

/Users/wuguanxun/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5)

In [44]:
centers = kmeans.cluster_centers_
from collections import Counter
dist = Counter(kmeans.labels_)
dist

Counter({1: 25, 3: 95, 0: 45, 4: 28, 2: 24})

<span style="color:black"><font size = 5> Task2.(b) : Label Prorpagation
    </span></font>

In [45]:
closest_per = 50
X_distance = kmeans.transform(X)
representative_X_index = np.argmin(X_distance,axis=0)
representative_X_image = X[representative_X_index]

In [46]:
X_cluster_dist = X_distance[np.arange(len(train_X)),kmeans.labels_]

for i in range(k):
    in_cluster = (kmeans.labels_ == i)#Choose the sample in the ith cluster
    cluster_distance = X_cluster_dist[in_cluster]#Filter the distance, leave only the distance of sample in ith cluster
    cutoff_distance = np.percentile(cluster_distance,closest_per) #calculate the margin value which distinguish whether or not the sample is the 20th nearest to the center 
    above_cutoff = (X_cluster_dist>cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

part_propagate = (X_cluster_dist != -1)
X_train_progated = train_X[part_propagate]
y_train_progated = train_y[part_propagate]

In [47]:
Unflattened_train_X = np.array([np.array(img.reshape((256,256,3))) for img in train_X])
Unflattened_test_X = np.array([np.array(img.reshape((256,256,3))) for img in test_X])
Unflattened_progated_X = np.array([np.array(img.reshape((256,256,3))) for img in X_train_progated])

In [48]:
mapping = {'Psilopogon_nuchalis':0,"Pycnonotus_sinensis":1}
df['species'].map(mapping)

0      0
1      0
2      0
3      0
4      0
      ..
306    1
307    1
308    1
309    1
310    1
Name: species, Length: 311, dtype: int64

<span style="color:black"><font size = 5>Task 2-(c.) The training data and the validation data are capable of passing through CNN from tensorflow</span></font>

In [49]:
train_ds = tf.data.Dataset.from_tensor_slices((Unflattened_train_X, np.array(train_y.map(mapping)))).batch(1)
train_ds_Kmeans = tf.data.Dataset.from_tensor_slices((Unflattened_progated_X, np.array(y_train_progated.map(mapping)))).batch(1)
valid_ds = tf.data.Dataset.from_tensor_slices((Unflattened_test_X,  np.array(test_y.map(mapping)))).batch(1)

In [50]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [73]:
#CNN using K-means selected data v.s. CNN the entire training set

num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu',input_shape = [256,256,3]),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),#Regularization
  tf.keras.layers.Dense(num_classes)
])


In [74]:
model.compile(
  optimizer='sgd',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model_Kmeans = model

In [75]:
from tensorflow import keras
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_v1.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(restore_best_weights = True)
history1 = model.fit(
          train_ds,
          validation_data = valid_ds,
          epochs=20,callbacks = [checkpoint_cb,early_stopping_cb],
          batch_size=1
)

Epoch 1/20
  5/217 [..............................] - ETA: 2s - loss: 0.9262 - accuracy: 0.4000     

2023-05-02 22:50:40.453130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


217/217 [==============================] - ETA: 0s - loss: 0.5655 - accuracy: 0.7465

2023-05-02 22:50:43.229080: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


217/217 [==============================] - 3s 14ms/step - loss: 0.5655 - accuracy: 0.7465 - val_loss: 0.4838 - val_accuracy: 0.9255
Epoch 2/20
217/217 [==============================] - 3s 14ms/step - loss: 0.5089 - accuracy: 0.7419 - val_loss: 0.4785 - val_accuracy: 0.9362


In [72]:
from tensorflow import keras
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_v2.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(restore_best_weights = True)
history2 = model_Kmeans.fit(
          train_ds_Kmeans,
          validation_data = valid_ds,
          epochs=20,
          callbacks = [checkpoint_cb,early_stopping_cb],
          batch_size=1
)

Epoch 1/20
110/110 [==============================] - 2s 16ms/step - loss: 3.7559e-09 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 2/20
110/110 [==============================] - 2s 16ms/step - loss: 3.7559e-09 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
